<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/weekly_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# MAKE SURE TO ENTER CORRECT GW LOWER DOWN IN CODE
import requests
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:



def get_team_name(id):
    general_information = requests.get(
        "https://fantasy.premierleague.com/api/bootstrap-static/"
    ).json()
    teams = general_information["teams"]
    for team in teams:
        if team["id"] == id:
            return team["name"]


def get_player_fixture_info(id):
    player_info = requests.get(
        f"https://fantasy.premierleague.com/api/element-summary/{id}/"
    ).json()
    fixture = player_info["fixtures"][0]
    home_team = get_team_name(fixture["team_h"])
    away_team = get_team_name(fixture["team_a"])
    kickoff_time = fixture["kickoff_time"]
    is_home = fixture["is_home"]
    return home_team, away_team, kickoff_time, is_home


def get_gameweek_result(id, gameweek):
    "returns a dictionary of the statistics from the gameweek"
    gameweek_results = requests.get(
        f"https://fantasy.premierleague.com/api/element-summary/{id}/"
    ).json()
    return gameweek_results["history"][gameweek - 1]


def get_player_position(element_type):
    general_information = requests.get(
        "https://fantasy.premierleague.com/api/bootstrap-static/"
    ).json()
    positions = general_information["element_types"]
    for position in positions:
        if position["id"] == element_type:
            return position["singular_name_short"]


general_info = requests.get(
    f"https://fantasy.premierleague.com/api/bootstrap-static/"
).json()
player_infos = general_info["elements"]
#***********************ENTER GW BELOW THAT JUST COMPLETED***********************
gameweek = 13
#***********************ENTER GW ABOVE THAT JUST COMPLETED***********************
"""gameweek_fixture = pd.read_csv(f"datasets/gw4.csv")
gameweek_fixture = pd.read_csv(
       f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/gws/gw4.csv"
    )

print(gameweek_fixture.columns)

def get_fixture(matchday, name):
    # load week's fixture
    global gameweek_fixture
    gameweek_fixture_ = gameweek_fixture[gameweek_fixture["name"] == name]
    #print(gameweek_fixture_["kickoff_time"].iloc[0])
    kickoff_time=gameweek_fixture_["kickoff_time"].iloc[0]
    team=gameweek_fixture_["team"].iloc[0]
    is_home=gameweek_fixture_["was_home"].iloc[0]
    opponent_team=get_team_name(gameweek_fixture_["opponent_team"].iloc[0])
    if is_home:
        home_team,away_team=team,opponent_team
    else:
        home_team,away_team=opponent_team,team

    return (
        home_team,
        away_team,
        kickoff_time,
        is_home
    )"""


gameweek_fixture = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/datasets/2024-25/fixtures/GW{gameweek}.csv")


def get_fixture(matchday, name):
    # load week's fixture
    global gameweek_fixture
    gameweek_fixture_ = gameweek_fixture[gameweek_fixture["name"] == name]
    return (
        gameweek_fixture_["home_team"].iloc[0],
        gameweek_fixture_["away_team"].iloc[0],
        gameweek_fixture_["kickoff_time"].iloc[0],
        gameweek_fixture_["is_home"].iloc[0],
    )

# week 6 results
all_players = []
for player_info in player_infos:
    try:
        #print(all_players)
        id = player_info["id"]
        print(id)
        name = player_info["first_name"] + " " + player_info["second_name"]
        team_id = player_info["team"]
        cost = player_info["now_cost"]
        element_type = player_info["element_type"]
        my_team = get_team_name(team_id)
        position = get_player_position(element_type)
        gameweek_result = get_gameweek_result(id, gameweek)
        home_team, away_team, kickoff_time, is_home = get_fixture(gameweek, name)
        #print(gameweek_result)
        gameweek_result["id"] = id
        gameweek_result["name"] = name
        gameweek_result["cost"] = cost
        gameweek_result["position"] = position
        gameweek_result["home_team"] = home_team
        gameweek_result["away_team"] = away_team
        gameweek_result["team_x"] = my_team
        gameweek_result["opponent_team"] = get_team_name(
            gameweek_result["opponent_team"]
        )
        all_players.append(gameweek_result)

    except IndexError:
        continue

# get x week result
print(all_players)
df = pd.DataFrame(all_players)
print(df)

1
2
3
4
5
6
7
8
9
12
13
15
16
17
18
20
21
22
23
24
25
73
186
578
597
633
638
639
657
658
659
662
665
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
233
569
598
640
641
656
679
59
60
61
62
63
64
65
66
67
68
69
70
71
72
74
75
76
77
78
79
80
81
83
84
85
152
580
592
599
617
680
687
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
314
338
602
634
649
661
674
113
114
115
116
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
595
600
601
615
618
628
629
663
668
681
685
150
151
153
154
155
157
158
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
187
189
386
560
575
581
621
11
159
190
193
194
195
196
197
199
200
201
202
203
205
206
207
208
209
210
211
212
213
214
215
216
445
585
644
645
646
647
650
666
676
156
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
23

In [3]:
df

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
0,1,129,West Ham,0,False,2024-11-30T17:30:00Z,2,5,13,0,...,1195,0,7,1,Fábio Ferreira Vieira,54,MID,West Ham,Arsenal,Arsenal
1,2,129,West Ham,1,False,2024-11-30T17:30:00Z,2,5,13,16,...,98296,1427,2426,2,Gabriel Fernando de Jesus,68,FWD,West Ham,Arsenal,Arsenal
2,3,129,West Ham,9,False,2024-11-30T17:30:00Z,2,5,13,45,...,2647421,193071,75444,3,Gabriel dos Santos Magalhães,62,DEF,West Ham,Arsenal,Arsenal
3,4,129,West Ham,6,False,2024-11-30T17:30:00Z,2,5,13,90,...,1106101,20558,124455,4,Kai Havertz,79,FWD,West Ham,Arsenal,Arsenal
4,5,129,West Ham,0,False,2024-11-30T17:30:00Z,2,5,13,0,...,3535,0,91,5,Karl Hein,40,GKP,West Ham,Arsenal,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,563,130,Bournemouth,1,True,2024-11-30T15:00:00Z,2,4,13,45,...,8370,719,383,563,Rodrigo Martins Gomes,52,MID,Wolves,Bournemouth,Wolves
612,564,130,Bournemouth,0,True,2024-11-30T15:00:00Z,2,4,13,0,...,7855,108,420,564,Santiago Bueno,43,DEF,Wolves,Bournemouth,Wolves
613,565,130,Bournemouth,0,True,2024-11-30T15:00:00Z,2,4,13,0,...,10772,490,992,565,Pablo Sarabia,51,MID,Wolves,Bournemouth,Wolves
614,566,130,Bournemouth,11,True,2024-11-30T15:00:00Z,2,4,13,90,...,297635,17460,56968,566,Jørgen Strand Larsen,56,FWD,Wolves,Bournemouth,Wolves


In [4]:
df[df["team_x"]=="Man City"]

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
367,340,125,Liverpool,0,False,2024-12-01T16:00:00Z,2,0,13,90,...,401534,2480,28649,340,Manuel Akanji,54,DEF,Liverpool,Man City,Man City
368,341,125,Liverpool,1,False,2024-12-01T16:00:00Z,2,0,13,90,...,102581,1083,2796,341,Nathan Aké,53,DEF,Liverpool,Man City,Man City
369,342,125,Liverpool,2,False,2024-12-01T16:00:00Z,2,0,13,90,...,413897,2943,53607,342,Bernardo Veiga de Carvalho e Silva,63,MID,Liverpool,Man City,Man City
370,343,125,Liverpool,0,False,2024-12-01T16:00:00Z,2,0,13,0,...,6548,59,165,343,Oscar Bobb,47,MID,Liverpool,Man City,Man City
371,344,125,Liverpool,0,False,2024-12-01T16:00:00Z,2,0,13,0,...,21871,296,354,344,Scott Carson,40,GKP,Liverpool,Man City,Man City
372,345,125,Liverpool,1,False,2024-12-01T16:00:00Z,2,0,13,11,...,420065,5596,17440,345,Kevin De Bruyne,94,MID,Liverpool,Man City,Man City
373,346,125,Liverpool,1,False,2024-12-01T16:00:00Z,2,0,13,33,...,155835,186,8258,346,Jérémy Doku,62,MID,Liverpool,Man City,Man City
374,347,125,Liverpool,0,False,2024-12-01T16:00:00Z,2,0,13,0,...,842305,2348,63664,347,Ederson Santana de Moraes,55,GKP,Liverpool,Man City,Man City
375,348,125,Liverpool,1,False,2024-12-01T16:00:00Z,2,0,13,78,...,515544,2586,65809,348,Phil Foden,91,MID,Liverpool,Man City,Man City
376,349,125,Liverpool,1,False,2024-12-01T16:00:00Z,2,0,13,11,...,21528,496,490,349,Jack Grealish,64,MID,Liverpool,Man City,Man City


In [5]:
df[df["team_x"]=="Liverpool"]

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
339,310,125,Man City,0,True,2024-12-01T16:00:00Z,2,0,13,0,...,550197,1767,14918,310,Alisson Ramses Becker,54,GKP,Liverpool,Man City,Liverpool
340,311,125,Man City,8,True,2024-12-01T16:00:00Z,2,0,13,72,...,2024095,22927,201259,311,Trent Alexander-Arnold,69,DEF,Liverpool,Man City,Liverpool
341,312,125,Man City,0,True,2024-12-01T16:00:00Z,2,0,13,0,...,4625,0,124,312,Stefan Bajčetić Maquieira,45,MID,Liverpool,Man City,Liverpool
342,313,125,Man City,0,True,2024-12-01T16:00:00Z,2,0,13,0,...,52285,4823,10629,313,Conor Bradley,47,DEF,Liverpool,Man City,Liverpool
343,315,125,Man City,0,True,2024-12-01T16:00:00Z,2,0,13,0,...,1719,0,28,315,Bobby Clark,45,MID,Liverpool,Man City,Liverpool
344,316,125,Man City,1,True,2024-12-01T16:00:00Z,2,0,13,17,...,260113,8072,23425,316,Darwin Núñez Ribeiro,71,FWD,Liverpool,Man City,Liverpool
345,317,125,Man City,0,True,2024-12-01T16:00:00Z,2,0,13,0,...,316564,298,12143,317,Diogo Teixeira da Silva,72,MID,Liverpool,Man City,Liverpool
346,318,125,Man City,0,True,2024-12-01T16:00:00Z,2,0,13,0,...,2060,0,47,318,Ben Doak,45,MID,Liverpool,Man City,Liverpool
347,319,125,Man City,1,True,2024-12-01T16:00:00Z,2,0,13,1,...,23667,232,311,319,Harvey Elliott,52,MID,Liverpool,Man City,Liverpool
348,320,125,Man City,0,True,2024-12-01T16:00:00Z,2,0,13,0,...,15706,426,676,320,Endo Wataru,47,MID,Liverpool,Man City,Liverpool


In [6]:
#save to folder
df.to_csv(f"/content/gdrive/MyDrive/fplpredict/datasets/2024-25/results/GW{gameweek}.csv")